<img style="float: left;" width="120" src="../images/kaggle.png">
<br><br><br>
<h2><center>House Prices: Advanced Regression Techniques</center></h2>

<h2><center> Modeling Regression  </center></h2>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score, roc_curve, auc
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from lightgbm import LGBMRegressor

sns.set_style("darkgrid")

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler,MinMaxScaler
#Import data
import pickle5 as pickle

In [2]:
file_train_X = open("../data/X_train_03_preprocess","rb")
file_test_X = open("../data/X_test_03_preprocess","rb")
file_train_y = open("../data/y_regression_train_03_preprocess","rb")


X = pickle.load(file_train_X)
y = pickle.load(file_train_y)
X_test = pickle.load(file_test_X)


X.isnull().sum().sum() #No missing values
X.shape, y.shape

((1460, 71), (1460,))

In [3]:
#y = np.log1p(y)

y.head()

Id
1    208500
2    181500
3    223500
4    140000
5    250000
Name: SalePrice, dtype: int64

In [4]:
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        X[c] = X[c].astype('category')

In [18]:
for c in X_test.columns:
    col_type = X_test[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        X_test[c] = X_test[c].astype('category')

In [5]:
import sklearn
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

In [6]:
grid_parameters = {}
pipes = {}
grids = {}
regressors = ['mlp','lgbm']

grid_parameters['mlp'] = {
    'MLPRegressor__max_iter': [5000],
    'MLPRegressor__activation' : ['relu'],
    'MLPRegressor__hidden_layer_sizes':[(10,10),(50,100,50),(100,100,100),(100,200,100)],}

grid_parameters['lgbm'] = {
    'lgbm__learning_rate' : [0.07],
    'lgbm__min_data_in_leaf' : list(range(1,35,5)),
    'lgbm__n_estimators' : list(range(10,500,50)),
    'lgbm__max_depth' : list(range(5,11,1)),}
    
    
pipes['mlp'] = Pipeline([('scaler',  MinMaxScaler()),
                        ('MLPRegressor', MLPRegressor(random_state = 303))])


pipes['lgbm'] = Pipeline([('scaler', RobustScaler()),('lgbm', LGBMRegressor(boosting_type='gbdt', objective='regression', 
                                                                         metric='neg_mean_squared_error'))])


In [7]:
cv = KFold(n_splits=5, shuffle=True, random_state=303)
grid = {}

for model in regressors:
    
    grid[model] = GridSearchCV (estimator = pipes[model],
                               param_grid = grid_parameters[model],
                               cv = cv,verbose=0,
                               n_jobs=-1)
    grid[model].fit(X, y)
    print(grid[model].best_score_)
    print(grid[model].best_params_)


C:\Users\pawel\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.7937290871526567
{'MLPRegressor__activation': 'relu', 'MLPRegressor__hidden_layer_sizes': (10, 10), 'MLPRegressor__max_iter': 5000}
0.8867080491195152
{'lgbm__learning_rate': 0.07, 'lgbm__max_depth': 10, 'lgbm__min_data_in_leaf': 11, 'lgbm__n_estimators': 460}


In [8]:
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error

cv = KFold(n_splits=5, shuffle=True, random_state=303)

def fit_score(y, y_pred):
    return mean_squared_error(y, y_pred)

def cv_score(model, scoring, X=X):
    
    rmse = cross_val_score(model, X, y,scoring = scoring, cv=cv)
    
    return (rmse)

In [14]:
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error

cv = KFold(n_splits=5, shuffle=True)
           
def fit_score(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_score(model, scoring, X=X):
    
    rmse = np.sqrt(-cross_val_score(model, X, y,scoring = scoring, cv=cv))
    
    return (rmse)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

for model in regressors:
    print(model)
    print("Fit Score - Mean Squared Error")
    print(fit_score(y, grid[model].predict(X)))
    print("\n")

In [ ]:
for model in regressors:
    print(model)
    score = cv_score(grid[model],"neg_mean_squared_error")
    print("CV Score - Mean Squared Error: {:.4f} ({:.4f})\n".format(score.mean(), score.std()) )
    print("\n")

In [23]:
submission = pd.read_csv("../data_regression/sample_submission.csv")
submission.iloc[:,1] = pd.Series(grid['lgbm'].predict(X_test))
#submission.iloc[:,1] = pd.Series(np.expm1(regressor.predict(X_test)))

submission.to_csv("../data_regression/submission.csv", index=False)
submission.head()
submission['SalePrice'].sum()

261061056.92820114

In [24]:
submission.head()

,Id,SalePrice
0,1461,125853.197028
1,1462,162627.547046
2,1463,182725.477182
3,1464,188589.692736
4,1465,179411.508456


<div style="text-align: right">Prepared by Pawel Sobieralski, 2020</div>